In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:10 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic/main amd64 Packages [48.3 kB]
G

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16Deliverable2").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Musical_Instruments_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   45610553| RMDCHWD0Y5OZ9|B00HH62VB6|     618218723|AGPtek® 10 Isolat...|Musical Instruments|          3|            0|          1|   N|                N|         Three Stars|Works very good, ...|2015-08-31 00:00:00|
|         US|   14640079| RZSL0BALIYUNU|B003LRN53I|     986692292|Sennheiser

In [28]:
# Create the vine_table DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RMDCHWD0Y5OZ9|          3|            0|          1|   N|                N|
| RZSL0BALIYUNU|          5|            0|          0|   N|                Y|
| RIZR67JKUDBI0|          3|            0|          1|   N|                Y|
|R27HL570VNL85F|          5|            0|          0|   N|                Y|
|R34EBU9QDWJ1GD|          5|            0|          0|   N|                Y|
|R1WCUI4Z1SIQEO|          5|            0|          0|   N|                N|
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|
|R3GYQ5W8JHP8SB|          5|            0|          0|   N|                Y|
|R30SHYQXGG5EYC|          5|            0|          0|   N|                Y|
|R14YLXA56NP51I|          5|            1|          1|   N|     

In [29]:
# 1. Filter data and create new DataFrame to retrieve all rows where the total_votes count is equal to or greater than 20
filtered_df = vine_df.filter('total_votes>=20')
filtered_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|  RMRD6SMF2AUQ|          3|            4|         21|   N|                N|
| RMPCXKWX3T57Y|          1|            1|         73|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|     

In [30]:
# 2. Filter the new DataFrame created in Step 1 and create a new DataFrame to retrieve all the rows where the number of helpful_votes
# divided by total_votes is equal to or greater than 50%
helpful_votes_df = filtered_df.filter(df['helpful_votes']/df['total_votes'] >= 0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [32]:
# Confirm the above table by adding a "helpful_votes/total_votes" column
helpful_votes_df2 = vine_df.withColumn('helpful_votes/total_votes', vine_df['helpful_votes']/vine_df['total_votes'] >= 0.5)
helpful_votes_df2 = helpful_votes_df2.filter(helpful_votes_df2['helpful_votes']/helpful_votes_df2['total_votes'] >= 0.5)
helpful_votes_df2.show()

+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|helpful_votes/total_votes|
+--------------+-----------+-------------+-----------+----+-----------------+-------------------------+
| RL5LNO26GAVJ1|          2|            3|          4|   N|                Y|                     true|
|R14YLXA56NP51I|          5|            1|          1|   N|                N|                     true|
|R2TZVLLTSHA07N|          5|            1|          1|   N|                Y|                     true|
|R2EFSVA4ZDWLN4|          5|            2|          2|   N|                Y|                     true|
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|                     true|
|R1G8LGJMUY210N|          5|            1|          2|   N|                Y|                     true|
|R1YJXOSQ36ZOR9|          5|            1|          1|   N|     

In [33]:
# 3. Filter the DataFrame created in Step 2 and create a new DataFrame that retrieves all the rows where a review was written as part of
# the Vine program (paid), vine == 'Y'
vine_paid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'Y')
vine_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1R9RU7JW0MFR2|          4|           20|         23|   Y|                N|
|R19EFYNN3W8Q07|          5|           26|         32|   Y|                N|
|R34DJ1R8AEU0SG|          5|           29|         35|   Y|                N|
|R25P5CXK5L9RHF|          5|          146|        161|   Y|                N|
|R2E9VZB3I4LSN5|          5|           55|         59|   Y|                N|
| RKYLHZL7EPELX|          4|           19|         25|   Y|                N|
|R1U13EKGQD3ZE6|          5|           22|         25|   Y|                N|
| RYW05F1MUEF01|          5|           87|        102|   Y|                N|
|R2SW4NXNO7HZJ5|          4|           28|         33|   Y|                N|
|R2016NFLSUR97Y|          2|           26|         37|   Y|     

In [34]:
# Repeat Step 3, but this time retrieve all the rows where the review was not part of the Vine program (unpaid), vine == 'N'
vine_unpaid_df = helpful_votes_df.filter(helpful_votes_df['vine'] == 'N')
vine_unpaid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2243Y3OD8U6KQ|          5|           47|         61|   N|                N|
|R2TGT0CDTCAAHW|          5|           21|         23|   N|                Y|
| RX4D22YSXEF4P|          1|           37|         38|   N|                Y|
|R3FL2NTLFUSPTQ|          5|           33|         37|   N|                N|
|R3QTP3YNZXAPPF|          3|           23|         24|   N|                Y|
|R36V6V42VN5AS5|          5|           34|         37|   N|                Y|
|R27LZWE27BJPOB|          5|           22|         23|   N|                N|
|R1P7GJ0IN2BRNH|          5|           37|         37|   N|                Y|
|R2R6JPF9KOD2HJ|          5|           19|         20|   N|                Y|
|R2J0ZZGFXKM8KR|          2|           21|         22|   N|     

In [35]:
# Step 5: Total Number of Reviews
# Get row count
rows = helpful_votes_df.count()
print(f"DataFrame Rows count : {rows}")

DataFrame Rows count : 14537


In [36]:
# Number of 5-star reviews
five_star_reviews = helpful_votes_df.filter(helpful_votes_df['star_rating'] == 5).count()
print(f"There are {five_star_reviews} 5-star reviews.")

There are 8246 5-star reviews.


In [37]:
# Number of paid and unpaid (overall)
paid = vine_paid_df.count()
unpaid = vine_unpaid_df.count()
print(f"There are {paid} paid reviews and {unpaid} unpaid reviews.")

There are 60 paid reviews and 14477 unpaid reviews.


In [38]:
# Number of paid and unpaid (five star reviews)
five_star_paid = vine_paid_df.filter(vine_paid_df['star_rating'] == 5).count()
five_star_unpaid = vine_unpaid_df.filter(vine_unpaid_df['star_rating'] == 5).count()
print(f"There are {five_star_paid} paid 5-star reviews and {five_star_unpaid} unpaid 5-star reviews.")

There are 34 paid 5-star reviews and 8212 unpaid 5-star reviews.


In [39]:
# Determine the percentage of 5-star reviews for the two types of review (paid vs unpaid)
five_star_paid_percent = five_star_paid/five_star_reviews * 100
five_star_unpaid_percent = five_star_unpaid/five_star_reviews * 100
print(f"{five_star_paid_percent}% of 5 star reviews were paid and {five_star_unpaid_percent}% were unpaid.")

0.41232112539413046% of 5 star reviews were paid and 99.58767887460587% were unpaid.
